<a id="Bi-LSTM CRF"></a>

# Bi-LSTM CRF Model

We will leverage a Bi-LSTM CRF model as the baseline tagger.

This notebook explores the construction of the tagger.

In [1]:
from pathlib import Path
import pandas as pd

In [2]:
ROOT_DIR = Path('notebooks/eda.ipynb').resolve().parents[2]
DATA_DIR = ROOT_DIR / "data"
PREPARED_DIR = DATA_DIR / "prepared"

In [3]:
from ast import literal_eval


df = pd.read_csv(PREPARED_DIR / "master.csv")
df["tags"] = df["tags"].apply(literal_eval)
df["single_tag"] = df["tags"].apply(lambda x: x[0])
df.head()

,sentence_num,word,start_idx,end_idx,tags,single_tag
0,0,This,0,4,[B-Temporal],B-Temporal
1,0,week,5,9,[I-Temporal],I-Temporal
2,0,sees,10,14,[O],O
3,0,the,15,18,[O],O
4,0,start,19,24,[O],O


In [7]:
words = set(list(df['word'].values))
words.add('PADword')
n_words = len(words)
print(f"There are {n_words} unique words")

There are 4332 unique words


In [8]:
tags = list(set(df["single_tag"].values))
n_tags = len(tags)
print(f"There are {n_tags} unique tags")

There are 29 unique tags


In [5]:
class SentenceGetter:
    """Iterator to get a sentence sequence and its BIO tags"""
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(),s["single_tag"].values.tolist())]
        self.grouped = self.data.groupby("sentence_num").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [10]:
getter = SentenceGetter(df)
sentences = getter.sentences
print(f"There are {len(sentences)} total sentence")

There are 1013 total sentence


In [11]:
# Make indices for ML modeling
words2index = {w:i for i,w in enumerate(words)}
tags2index = {t:i for i,t in enumerate(tags)}

In [12]:
from tensorflow.keras.utils import pad_sequences

max_len = 50
y = [[tags2index[w[1]] for w in s] for s in sentences]

y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tags2index["O"])

In [13]:
y.shape

(1013, 50)

In [15]:
max_len = 50
X = [[words2index[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)

In [16]:
import tensorflow as tf

x_tensor = tf.convert_to_tensor(X)
y_tensor = tf.convert_to_tensor(y)

ds = tf.data.Dataset.from_tensor_slices((x_tensor, y_tensor))

BATCH_SIZE = 64

2022-11-10 10:27:19.552134: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2022-11-10 10:27:19.552182: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: enyquist-X399-DESIGNARE-EX
2022-11-10 10:27:19.552193: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: enyquist-X399-DESIGNARE-EX
2022-11-10 10:27:19.552360: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.85.2
2022-11-10 10:27:19.552391: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 520.61.5
2022-11-10 10:27:19.552400: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 520.61.5 does not match DSO version 510.85.2 -- cannot find working devices in this configuration
2022-11-10 10:27:19.553048: I tensorflow/core/platform/cpu_f

In [17]:
def get_dataset_partitions_tf(
    ds, 
    ds_size, 
    train_split=0.8, 
    val_split=0.1, 
    test_split=0.1, 
    shuffle=True, 
    shuffle_size=1000
):
    assert (train_split + test_split + val_split) == 1

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=42)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds.batch(BATCH_SIZE), val_ds.batch(BATCH_SIZE), test_ds.batch(BATCH_SIZE)

In [18]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(
    ds=ds,
    ds_size=X.shape[0],
)

In [19]:
import io
from pathlib import Path
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, TimeDistributed, Input, Dropout
from tensorflow.keras.optimizers import Adam

In [20]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow_addons.text import crf_log_likelihood, crf_decode

"""
Credit:
https://github.com/ngoquanghuy99/POS-Tagging-BiLSTM-CRF
"""

class CRF(L.Layer):
    def __init__(self,
                 output_dim,
                 sparse_target=True,
                 **kwargs):
        """    
        Args:
            output_dim (int): the number of labels to tag each temporal input.
            sparse_target (bool): whether the the ground-truth label represented in one-hot.
        Input shape:
            (batch_size, sentence length, output_dim)
        Output shape:
            (batch_size, sentence length, output_dim)
        """
        super(CRF, self).__init__(**kwargs)
        self.output_dim = int(output_dim) 
        self.sparse_target = sparse_target
        self.input_spec = L.InputSpec(min_ndim=3)
        self.supports_masking = False
        self.sequence_lengths = None
        self.transitions = None

    def build(self, input_shape):
        assert len(input_shape) == 3
        f_shape = tf.TensorShape(input_shape)
        input_spec = L.InputSpec(min_ndim=3, axes={-1: f_shape[-1]})

        if f_shape[-1] is None:
            raise ValueError('The last dimension of the inputs to `CRF` '
                             'should be defined. Found `None`.')
        if f_shape[-1] != self.output_dim:
            raise ValueError('The last dimension of the input shape must be equal to output'
                             ' shape. Use a linear layer if needed.')
        self.input_spec = input_spec
        self.transitions = self.add_weight(name='transitions',
                                           shape=[self.output_dim, self.output_dim],
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.built = True

    def compute_mask(self, inputs, mask=None):
        # Just pass the received mask from previous layer, to the next layer or
        # manipulate it if this layer changes the shape of the input
        return mask

    def call(self, inputs, sequence_lengths=None, training=None, **kwargs):
        sequences = tf.convert_to_tensor(inputs, dtype=self.dtype)
        if sequence_lengths is not None:
            assert len(sequence_lengths.shape) == 2
            assert tf.convert_to_tensor(sequence_lengths).dtype == 'int32'
            seq_len_shape = tf.convert_to_tensor(sequence_lengths).get_shape().as_list()
            assert seq_len_shape[1] == 1
            self.sequence_lengths = K.flatten(sequence_lengths)
        else:
            self.sequence_lengths = tf.ones(tf.shape(inputs)[0], dtype=tf.int32) * (
                tf.shape(inputs)[1]
            )

        viterbi_sequence, _ = crf_decode(sequences,
                                         self.transitions,
                                         self.sequence_lengths)
        output = K.one_hot(viterbi_sequence, self.output_dim)
        return K.in_train_phase(sequences, output)

    @property
    def loss(self):
        def crf_loss(y_true, y_pred):
            y_pred = tf.convert_to_tensor(y_pred, dtype=self.dtype)
            log_likelihood, self.transitions = crf_log_likelihood(
                y_pred,
                tf.cast(K.argmax(y_true), dtype=tf.int32) if self.sparse_target else y_true,
                self.sequence_lengths,
                transition_params=self.transitions,
            )
            return tf.reduce_mean(-log_likelihood)
        return crf_loss

    @property
    def accuracy(self):
        def viterbi_accuracy(y_true, y_pred):
            # -1e10 to avoid zero at sum(mask)
            mask = K.cast(
                K.all(K.greater(y_pred, -1e10), axis=2), K.floatx())
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            y_pred, _ = crf_decode(y_pred, self.transitions, sequence_lengths)
            if self.sparse_target:
                y_true = K.argmax(y_true, 2)
            y_pred = K.cast(y_pred, 'int32')
            y_true = K.cast(y_true, 'int32')
            corrects = K.cast(K.equal(y_true, y_pred), K.floatx())
            return K.sum(corrects * mask) / K.sum(mask)
        return viterbi_accuracy

    @property
    def f1(self):
        def crf_f1(y_true, y_pred):
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            y_pred, _ = crf_decode(y_pred, self.transitions, sequence_lengths)
            true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
            possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
            predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
            precision = true_positives / (predicted_positives + K.epsilon())
            recall = true_positives / (possible_positives + K.epsilon())
            f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
            return f1_val
        return crf_f1

    def compute_output_shape(self, input_shape):
        tf.TensorShape(input_shape).assert_has_rank(3)
        return input_shape[:2] + (self.output_dim,)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'sparse_target': self.sparse_target,
            'supports_masking': self.supports_masking,
            'transitions': K.eval(self.transitions)
        }
        base_config = super(CRF, self).get_config()
        return dict(base_config, **config)


In [21]:
EMBEDDING_DIR = DATA_DIR / "embeddings"

In [22]:
"""
Credit:
https://github.com/ngoquanghuy99/POS-Tagging-BiLSTM-CRF
"""

def create_model(
    vocab_size,
    max_length,
    embedding_dim,
    word_index,
    tag_index
):

    embeddings_index = {}
    with io.open(EMBEDDING_DIR / 'glove.6B.100d.txt', 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            curr_word = values[0]
            coefs = np.asarray(values[1:], dtype='float64')
            embeddings_index[curr_word] = coefs
        embeddings_matrix = np.zeros((vocab_size, embedding_dim))
        for word, i in word_index.items():
            if i > vocab_size:
                continue
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embeddings_matrix[i] = embedding_vector

    model = Sequential()
    model.add(
        Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            input_length=max_length,
            weights=[embeddings_matrix],
            mask_zero=True
        )
    )

    model.add(
        Bidirectional(
            LSTM(
                units=embedding_dim, return_sequences=True, recurrent_dropout=0.01
            )
        )
    )
    
    model.add(TimeDistributed(Dense(len(tag_index))))

    crf = CRF(len(tag_index), sparse_target=False)
    model.add(crf)

    model.compile(optimizer="adam", loss=crf.loss, metrics=[crf.accuracy])
    model.summary()
    return model

In [23]:
model = create_model(
    vocab_size=len(words2index),
    max_length=50,
    embedding_dim=100,
    word_index=words2index,
    tag_index=tags2index
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 100)           433200    
                                                                 
 bidirectional (Bidirectiona  (None, 50, 200)          160800    
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 50, 29)           5829      
 ibuted)                                                         
                                                                 
 crf (CRF)                   (None, 50, 29)            841       
                                                                 
Total params: 600,670
Trainable params: 600,670
Non-trainable params: 0
_________________________________________________________________


In [24]:
import shutil

log_dir = ROOT_DIR / "models/logs/"

if any(log_dir.iterdir()):
    for i in log_dir.glob("**/*"):
        if i.is_dir():
            shutil.rmtree(i)
        else:
            i.unlink()

In [25]:
callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=3)
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, 
    histogram_freq=1,
    profile_batch="1,20"
)


history = model.fit(
    train_ds,
    epochs=100, 
    verbose=1,
    validation_data=val_ds,
    callbacks=[callback, tensorboard_callback],
)

Epoch 1/100


2022-11-10 10:29:56.585248: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-10 10:29:56.585293: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-11-10 10:29:56.586009: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2022-11-10 10:29:56.616318: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-10 10:29:56.616352: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-11-10 10:30:01.030429: W tensorflow/core/common_runtime/forward_type_inference.cc:231] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}
 is neither a subtype nor a supertype of the combined

13/13 [==============================] - 9s 191ms/step - loss: 128.2678 - viterbi_accuracy: 0.5286 - val_loss: 139.7882 - val_viterbi_accuracy: 0.4347
Epoch 2/100
 6/13 [============>.................] - ETA: 0s - loss: 54.5011 - viterbi_accuracy: 0.8465

2022-11-10 10:30:06.553633: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-11-10 10:30:08.538850: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2022-11-10 10:30:11.627919: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /home/enyquist/repos/RLNER/models/logs/plugins/profile/2022_11_10_10_30_08

2022-11-10 10:30:14.725803: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to /home/enyquist/repos/RLNER/models/logs/plugins/profile/2022_11_10_10_30_08/enyquist-X399-DESIGNARE-EX.trace.json.gz
2022-11-10 10:30:16.174196: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /home/enyquist/repos/RLNER/models/logs/plugins/profile/2022_11_10_10_30_08

2022-11-10 10:30:16.174331: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to /home/enyquist

13/13 [==============================] - 12s 991ms/step - loss: 49.0816 - viterbi_accuracy: 0.8460 - val_loss: 139.0940 - val_viterbi_accuracy: 0.4413
Epoch 3/100
13/13 [==============================] - 1s 107ms/step - loss: 40.6658 - viterbi_accuracy: 0.8377 - val_loss: 138.6207 - val_viterbi_accuracy: 0.4368
Epoch 4/100
13/13 [==============================] - 1s 106ms/step - loss: 35.6045 - viterbi_accuracy: 0.8443 - val_loss: 138.8639 - val_viterbi_accuracy: 0.4397
Epoch 5/100
13/13 [==============================] - 1s 108ms/step - loss: 33.0754 - viterbi_accuracy: 0.8438 - val_loss: 138.1800 - val_viterbi_accuracy: 0.8336
Epoch 6/100
13/13 [==============================] - 1s 110ms/step - loss: 29.7640 - viterbi_accuracy: 0.8488 - val_loss: 137.8049 - val_viterbi_accuracy: 0.8417
Epoch 7/100
13/13 [==============================] - 1s 104ms/step - loss: 28.0674 - viterbi_accuracy: 0.8490 - val_loss: 136.9574 - val_viterbi_accuracy: 0.8660
Epoch 8/100
13/13 [====================

In [26]:
model.evaluate(test_ds)

2/2 [==============================] - 0s 40ms/step - loss: 124.3278 - viterbi_accuracy: 0.9909


[124.32781982421875, 0.9909210205078125]

The baseline Bi-LSTM CRF model has compiled, trained, and tested on the Re3d dataset successfully! Achieving 99.09% Accuracy on the test set, which is pretty good, as only ~68% of the tags are "O" tags.